# VV Correlation Heatmaps

Ravi Mandla

Code used to create correlation heatmaps of VV RNA-seq data

Code was taken from http://www.sthda.com/english/wiki/ggplot2-quick-correlation-matrix-heatmap-r-software-and-data-visualization

In [1]:
library(reshape2)
library(ggplot2)
library(readxl)
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ tibble  3.0.1     ✔ dplyr   1.0.0
✔ tidyr   1.1.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0
✔ purrr   0.3.4     

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



Functions

In [2]:
# Get lower triangle of the correlation matrix
  get_lower_tri<-function(cormat){
    cormat[upper.tri(cormat)] <- NA
    return(cormat)
  }
  # Get upper triangle of the correlation matrix
  get_upper_tri <- function(cormat){
    cormat[lower.tri(cormat)]<- NA
    return(cormat)
  }

reorder_cormat <- function(cormat){
# Use correlation between variables as distance
dd <- as.dist((1-cormat)/2)
hc <- hclust(dd)
cormat <-cormat[hc$order, hc$order]
}

Import data

In [3]:
data <- read_tsv("~/Downloads/normalized_counts.tsv")

Parsed with column specification:
cols(
  ID = col_character(),
  Name = col_character(),
  VV.Red._ = col_double(),
  VV.Red.P2 = col_double(),
  VV.Red.P4 = col_double(),
  VV.RedGreen._ = col_double(),
  VV.RedGreen.P2 = col_double(),
  VV.RedGreen.P4 = col_double()
)



In [63]:
norm_genename = data %>% select(3:8)

In [64]:
rownames(norm_genename) = data$...1

Warning message:
“Unknown or uninitialised column: `...1`.”


Make heatmap

In [65]:
cormat <- round(cor(norm_genename),2)

In [66]:
upper_tri <- get_upper_tri(cormat)

In [67]:
melted_cormat <- melt(upper_tri, na.rm = TRUE)

In [68]:
# Reorder the correlation matrix
cormat <- reorder_cormat(cormat)
upper_tri <- get_upper_tri(cormat)
# Melt the correlation matrix
melted_cormat <- melt(upper_tri, na.rm = TRUE)
# Create a ggheatmap
ggheatmap <- ggplot(melted_cormat, aes(Var2, Var1, fill = value))+
 geom_tile(color = "white")+
 scale_fill_gradient2(low = "blue", high = "red", mid = "white", 
   midpoint = 0, limit = c(-1,1), space = "Lab", 
    name="Pearson\nCorrelation") +
  theme_minimal()+ # minimal theme
 theme(axis.text.x = element_text(angle = 45, vjust = 1, 
    size = 12, hjust = 1))+
 coord_fixed()

In [69]:
png(file='corr-heatmap-rm.png')
ggheatmap + 
geom_text(aes(Var2, Var1, label = value), color = "black", size = 4) +
theme(
  axis.title.x = element_blank(),
  axis.title.y = element_blank(),
  panel.grid.major = element_blank(),
  panel.border = element_blank(),
  panel.background = element_blank(),
  axis.ticks = element_blank(),
  legend.justification = c(1, 0),
  legend.position = c(0.6, 0.7),
  legend.direction = "horizontal")+
  guides(fill = guide_colorbar(barwidth = 7, barheight = 1,
                title.position = "top", title.hjust = 0.5))
dev.off()

png 
  2

This heatmap is not interesting, so I will make one with only DE genes also.

In [12]:
fdr = read.table("~/Downloads/deseq2_res.csv",sep=',',header=TRUE)

In [13]:
sig = na.omit(fdr[fdr$padj < .05, ])

In [14]:
sigensembl = sig$"Ensembl"

In [15]:
normvals = read_tsv("~/Downloads/normalized_counts.tsv")

Parsed with column specification:
cols(
  ID = col_character(),
  Name = col_character(),
  VV.Red._ = col_double(),
  VV.Red.P2 = col_double(),
  VV.Red.P4 = col_double(),
  VV.RedGreen._ = col_double(),
  VV.RedGreen.P2 = col_double(),
  VV.RedGreen.P4 = col_double()
)



In [16]:
vals = normvals %>% select(1,3:8)

In [17]:
sig_normvals = filter(vals, ID %in% sigensembl)

In [18]:
sig_normvals

ID,VV.Red._,VV.Red.P2,VV.Red.P4,VV.RedGreen._,VV.RedGreen.P2,VV.RedGreen.P4
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSMUSG00000025900,106.716190,42.443705,117.0289701,15.9305541,4.983390,23.495670
ENSMUSG00000025905,0.000000,0.000000,0.0000000,1.9913193,2.990034,75.521798
ENSMUSG00000025909,8.586360,70.032113,19.9382690,1.9913193,0.000000,0.000000
ENSMUSG00000016918,371.666730,808.552583,1869.8628780,247.9192481,294.019996,469.913408
ENSMUSG00000102664,46.611669,87.009596,52.0128756,93.5920053,242.192742,194.678412
ENSMUSG00000025932,363.080370,354.404938,259.1974968,70.6918338,20.930237,54.543521
ENSMUSG00000054493,20.852589,16.977482,23.4057940,0.0000000,2.990034,0.000000
ENSMUSG00000025930,4.906491,11.672019,61.5485695,0.0000000,0.000000,0.000000
ENSMUSG00000102937,0.000000,2.122185,0.8668813,6.9696174,36.877084,45.313079


In [19]:
sig_normvals = select(sig_normvals,2:7)

In [20]:
cormat <- round(cor(sig_normvals),2)
upper_tri <- get_upper_tri(cormat)
melted_cormat <- melt(upper_tri, na.rm = TRUE)

# Reorder the correlation matrix
cormat <- reorder_cormat(cormat)
upper_tri <- get_upper_tri(cormat)
# Melt the correlation matrix
melted_cormat <- melt(upper_tri, na.rm = TRUE)
# Create a ggheatmap
ggheatmap <- ggplot(melted_cormat, aes(Var2, Var1, fill = value))+
 geom_tile(color = "white")+
 scale_fill_gradient2(low = "blue", high = "red", mid = "white", 
   midpoint = 0, limit = c(-1,1), space = "Lab", 
    name="Pearson\nCorrelation") +
  theme_minimal()+ # minimal theme
 theme(axis.text.x = element_text(angle = 45, vjust = 1, 
    size = 12, hjust = 1))+
 coord_fixed()

png(file='corr-heatmap-rm-de.png')
ggheatmap + 
geom_text(aes(Var2, Var1, label = value), color = "black", size = 4) +
theme(
  axis.title.x = element_blank(),
  axis.title.y = element_blank(),
  panel.grid.major = element_blank(),
  panel.border = element_blank(),
  panel.background = element_blank(),
  axis.ticks = element_blank(),
  legend.justification = c(1, 0),
  legend.position = c(0.6, 0.7),
  legend.direction = "horizontal")+
  guides(fill = guide_colorbar(barwidth = 7, barheight = 1,
                title.position = "top", title.hjust = 0.5))
dev.off()

png 
  2